In [15]:
def get_embed_code(video_url):
    import re
    video_id = re.search(r"v=([^&]+)", video_url)
    if video_id:
        return f'<iframe width="560" height="315" src="https://www.youtube.com/embed/{video_id.group(1)}" frameborder="0" allowfullscreen></iframe>'
    return None
url = "https://www.youtube.com/watch?v=l-DM86KmhHQ&t=3308s"
print(get_embed_code(url))

<iframe width="560" height="315" src="https://www.youtube.com/embed/l-DM86KmhHQ" frameborder="0" allowfullscreen></iframe>


In [16]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [17]:
os.environ['TAVILY_API_KEY']=os.getenv("TAVILY_API_KEY")

In [18]:
from langchain_tavily import TavilySearch

In [19]:
tool = TavilySearch(
    max_results=50)

In [20]:
response=tool.invoke({'query':"give me links of PCA videos in youtube. you must provide only youtube urls. url format: https://www.youtube.com/"})

In [21]:
def get_youtube_urls(query):
    tool = TavilySearch(max_results=50)
    response=tool.invoke({'query':f"give me links of {query} videos in youtube. you must provide only youtube urls. url format: https://www.youtube.com/"})
    youtube_urls=[]
    for i in response["results"]:
        if is_youtube_url(i['url']):
            youtube_urls.append(i['url'])
    return youtube_urls

In [22]:
type(response)

dict

In [23]:
import re

def is_youtube_url(url: str) -> bool:
    """
    Checks whether the given URL is a valid YouTube video URL.
    """
    youtube_patterns = [
        r'^https?://(www\.)?youtube\.com/watch\?v=.+',
        r'^https?://youtu\.be/.+'
    ]
    
    return any(re.match(pattern, url) for pattern in youtube_patterns)


In [24]:
youtube_urls=[]
for i in response["results"]:
    if is_youtube_url(i['url']):
        youtube_urls.append(i['url'])

In [25]:
len(response["results"])

50

In [26]:
youtube_urls

['https://www.youtube.com/watch?v=-Zmg_Q85Wt4',
 'https://www.youtube.com/watch?v=KA3csdPvp2E',
 'https://www.youtube.com/watch?v=MN3nup1n0sU']

In [27]:
embeded_codes=[]
for i in youtube_urls:
    embeded_codes.append(get_embed_code(i))

In [28]:
embeded_codes

['<iframe width="560" height="315" src="https://www.youtube.com/embed/-Zmg_Q85Wt4" frameborder="0" allowfullscreen></iframe>',
 '<iframe width="560" height="315" src="https://www.youtube.com/embed/KA3csdPvp2E" frameborder="0" allowfullscreen></iframe>',
 '<iframe width="560" height="315" src="https://www.youtube.com/embed/MN3nup1n0sU" frameborder="0" allowfullscreen></iframe>']

In [29]:
from langchain_groq import ChatGroq
llm=ChatGroq(model="gemma2-9b-it")

In [30]:
llm.invoke("hello")

AIMessage(content='Hello! 👋\n\nHow can I help you today? 😊\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 11, 'total_tokens': 26, 'completion_time': 0.027272727, 'prompt_time': 0.001919026, 'queue_time': 0.23121692300000002, 'total_time': 0.029191753}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-e43a3050-e379-4450-a903-4302253634e1-0', usage_metadata={'input_tokens': 11, 'output_tokens': 15, 'total_tokens': 26})

In [31]:
from pydantic import BaseModel,Field

In [35]:
class QuizFormat(BaseModel):
    Question:str =Field(description="Question")
    A:str = Field(description="first option")
    B: str = Field(description="second option")
    C: str = Field(description="Third option")
    D: str = Field(description="fourth option")
    Answer:str = Field(description="Answer")
    Explanation:str = Field(description="Explantion of the answer in two lines")

In [36]:
quiz_llm=llm.with_structured_output(QuizFormat)

In [38]:
quiz_llm.invoke("Ask 10 questions about machine learning")

QuizFormat(Question='What is machine learning?', A='A type of artificial intelligence that involves training computers to learn from data without explicit programming.', B='A programming language designed for building AI models.', C='A method for analyzing large datasets.', D='A type of database used to store machine learning models.', Answer='A', Explanation='Machine learning algorithms learn patterns and relationships from data, enabling them to make predictions or decisions on new, unseen data.')

In [ ]:
from pydantic import BaseModel, Field
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableLambda
from typing import List

# Define the QuizFormat structure
class QuizFormat(BaseModel):
    Question: str = Field(description="Question")
    A: str = Field(description="first option")
    B: str = Field(description="second option")
    C: str = Field(description="Third option")
    D: str = Field(description="fourth option")
    Answer: str = Field(description="Correct answer with option letter and full text")
    Explanation: str = Field(description="Explanation of the answer in two lines")

# Create the prompt template
prompt_template = PromptTemplate.from_template("""
You are a professional AI quiz generator. Your job is to generate high-quality multiple-choice questions for students.

Guidelines:
- Number of questions: {number_of_questions}
- Topic: {topic}
- Difficulty Level: {level} (easy, medium, hard, or mixed — if "mixed", include a blend)
- Each question should follow this format:
    Q. [Question]
    A. Option A
    B. Option B
    C. Option C
    D. Option D
    Correct Answer: [Option Letter] [Full text of correct option]
    Explanation: [Short and clear explanation of why this option is correct]
- Only one correct answer per question.
- Keep questions unique and non-repetitive.
- Explanations should be simple, educational, and not more than two lines.

Generate the next question now.
""")

# Assume you already have a loaded LLM (e.g., from ChatOpenAI or ChatGroq)
# If not:
# from langchain.chat_models import ChatOpenAI
# llm = ChatOpenAI(model_name="gpt-3.5-turbo")

# Wrap the LLM with structured output
quiz_llm = llm.with_structured_output(QuizFormat)

# Chain to generate multiple questions
def generate_quiz(topic: str, level: str, number_of_questions: int) -> List[QuizFormat]:
    results = []
    for _ in range(number_of_questions):
        input_vars = {
            "topic": topic,
            "level": level,
            "number_of_questions": number_of_questions  # just passed for prompt consistency
        }
        question = quiz_llm.invoke(prompt_template.format(**input_vars))
        results.append(question)
    return results

# Example usage
quiz_questions = generate_quiz(topic="Linear Regression", level="hard", number_of_questions=10)

# Print the results
for i, q in enumerate(quiz_questions, 1):
    print(f"\nQ{i}. {q.Question}")
    print(f"A. {q.A}")
    print(f"B. {q.B}")
    print(f"C. {q.C}")
    print(f"D. {q.D}")
    print(f"Correct Answer: {q.Answer}")
    print(f"Explanation: {q.Explanation}")



Q1. In linear regression, what does the coefficient of determination (R-squared) represent?
A. The slope of the regression line
B. The correlation between the independent and dependent variables
C. The proportion of variance in the dependent variable explained by the independent variable
D. The y-intercept of the regression line
Correct Answer: C The proportion of variance in the dependent variable explained by the independent variable
Explanation: R-squared indicates how well the linear model fits the data. A higher R-squared value means a better fit.

Q2. In a linear regression model, what does the R-squared value represent?
A. The slope of the regression line
B. The y-intercept of the regression line
C. The proportion of the variance in the dependent variable that is explained by the independent variable
D. The strength of the linear relationship between the independent and dependent variables
Correct Answer: C. The proportion of the variance in the dependent variable that is expla